
Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

Задание 2.
найти фичи с наибольшей значимостью, и вывести их

Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

from nltk import tokenize as tknz
from nltk import ngrams

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import os
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings("ignore")

In [2]:
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evamelissatasdemir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
punctuation

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/evamelissatasdemir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/evamelissatasdemir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/evamelissatasdemir/nltk_data...


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
noise = stopwords.words('russian') + list(punctuation)

In [6]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [7]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [10]:
df.head(10)

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive
5,ну любишь или нет? — Я не знаю кто ты бля:D ht...,positive
6,"RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",positive
7,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,positive
8,Поприветствуем моего нового читателя @Alexey17...,positive
9,Теперь у меня есть частичка Сиднея :) #Sydney ...,positive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226834 entries, 0 to 111922
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 5.2+ MB


In [16]:
def remove_word_user(input_txt):
    pattern = "@[\w]*"
    lst = re.findall(pattern, input_txt)
    result = input_txt
    for el in lst:
        result = re.sub(el, ' ', result)
    return result

df.text[0]

0      хоть я и школота, но поверь, у нас то же сам...
0    на работе был полный пиддес :| и так каждое за...
Name: text, dtype: object

In [15]:
df['text'] = np.vectorize(remove_word_user)(df['text'])
df.head()

,text,label
0,"хоть я и школота, но поверь, у нас то же сам...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT : Ну ты идиотка) я испугалась за тебя!!!,positive
3,"RT : ""Кто то в углу сидит и погибает от голод...",positive
4,"Вот что значит страшилка :D\nНо блин,посмотр...",positive


In [7]:
#x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [18]:
def remove_noise(token, noise):
    """ 
    Removing noise from tokens.
    """
    remove_sw = [word for word in token if not word in noise]
    return  remove_sw

In [19]:
def get_lemmatizer(words, lemmatizer, pos):
    """
    lemmitization
    """
    lemmas = []
    for word in words:
        lemmas.append(lemmatizer.lemmatize(word, pos = nltk.corpus.wordnet.VERB) )
         
    return lemmas  

In [39]:
%%time

# Tokenizing
# Removing stop-words and punctuation
# Lemmatization

df['text_tokens'] = df['text'].apply(tknz.word_tokenize)
df['text_tokens'] = df['text_tokens'].apply(remove_noise, noise=noise)
df['text_tokens'] = \

        df['text_tokens'].apply(get_lemmatizer, lemmatizer = WordNetLemmatizer(), pos = wordnet.VERB)
df.head(3)

SyntaxError: invalid syntax (<unknown>, line 8)

In [ ]:
count_vect_corp = CountVectorizer(analyzer='word', 
                                  token_pattern=r'\w{1,}')

tfidf_vec_corp = TfidfVectorizer(analyzer='word', 
                                 token_pattern=r'\w{1,}')

count_vect_corp_mod = CountVectorizer(max_df=0.9, 
                                      max_features = 1000, 
                                      stop_words='russian', 
                                      analyzer='word', 
                                      token_pattern=r'\w{1,}')

tfidf_vec_corp_mod = TfidfVectorizer(max_df=0.9, 
                                     max_features = 1000, 
                                     stop_words='russian', 
                                     analyzer='word', 
                                     token_pattern=r'\w{1,}')